In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [2]:
import spacy

In [3]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [4]:
nlp.max_length = 1198623

In [5]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [6]:
d = read_file('C:\\Users\\eobuobi\\trial\\nlp_project\\data\\raw\\moby_dick_four_chapters.txt')

In [7]:
tokens = separate_punc(d)

In [8]:
len(tokens)

11338

In [9]:
#25words --> network will predict #26

In [10]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [11]:
type(text_sequences)

list

In [12]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [13]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [14]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [15]:
from keras.preprocessing.text import Tokenizer

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [17]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [18]:
#id to a word
# sequences[1]

In [19]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [20]:
#tokenizer.word_counts

In [21]:
vocabulary_size = len(tokenizer.word_counts)

In [22]:
vocabulary_size

2717

In [23]:
type(sequences)

list

In [24]:
import numpy as np

In [25]:
sequences = np.array(sequences)

In [26]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [27]:
from keras.utils import to_categorical

In [28]:
X = sequences[:,:-1]

In [29]:
y = sequences[:,-1]

In [30]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [31]:
seq_len= X.shape[1]

In [32]:
X.shape

(11312, 25)

In [33]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [34]:
def create_model(vocabulary_size,seq_len):
    
    model = Sequential()
    model.add(Embedding(vocabulary_size,seq_len, input_length=seq_len))
    model.add(LSTM(seq_len*2,return_sequences=True))
    model.add(LSTM(seq_len*2))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    
    return model

In [35]:
model = create_model(vocabulary_size+1,seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67950     
                                                                 
 lstm (LSTM)                 (None, 25, 50)            15200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 2718)              138618    
                                                                 
Total params: 244,518
Trainable params: 244,518
Non-trainable params: 0
_________________________________________________________________


In [36]:
from pickle import dump,load

In [37]:
model.fit(X,y,batch_size=128, epochs=350, verbose= 1)

Epoch 1/350
89/89 [==============================] - 4s 10ms/step - loss: 6.9773 - accuracy: 0.0426
Epoch 2/350
89/89 [==============================] - 1s 10ms/step - loss: 6.3745 - accuracy: 0.0529
Epoch 3/350
89/89 [==============================] - 1s 10ms/step - loss: 6.3450 - accuracy: 0.0529
Epoch 4/350
89/89 [==============================] - 1s 10ms/step - loss: 6.2839 - accuracy: 0.0529
Epoch 5/350
89/89 [==============================] - 1s 9ms/step - loss: 6.1760 - accuracy: 0.0529
Epoch 6/350
89/89 [==============================] - 1s 9ms/step - loss: 6.1111 - accuracy: 0.0530
Epoch 7/350
89/89 [==============================] - 1s 9ms/step - loss: 6.0571 - accuracy: 0.0526
Epoch 8/350
89/89 [==============================] - 1s 9ms/step - loss: 5.9786 - accuracy: 0.0551
Epoch 9/350
89/89 [==============================] - 1s 9ms/step - loss: 5.8964 - accuracy: 0.0598
Epoch 10/350
89/89 [==============================] - 1s 9ms/step - loss: 5.8366 - accuracy: 0.0633
Epoch

In [38]:
model.save('my_mobydick_model.h5')

In [39]:
dump(tokenizer,open('my_simpletokenizer', 'wb'))

In [41]:
from random import randint
from pickle import load
from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences

In [66]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = np.argmax(model.predict(pad_encoded, verbose=0)[0], axis=0)
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [67]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [68]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [69]:
random_seed_text = text_sequences[random_pick]

In [70]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [71]:
seed_text = ' '.join(random_seed_text)

In [72]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [73]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

"to be afraid of him then to spoil the hilarity of his shipmates and presently the plane take in the year in his chest he save the good hunch or a dream but had are her kept it i wo n't see him piled whoever and put more crowds for"